In [1]:
import os
import numpy as np
import cv2
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier
import pickle
from tqdm import tqdm

C:\Users\oknol\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\oknol\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\oknol\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# data path
data_dir = './clf-data'
categories = ['empty', 'not_empty']  # 0 = empty, 1 = notempty

X = []
y = []

In [3]:
# uploading images and taking them into the process
for label, category in enumerate(categories):
    folder_path = os.path.join(data_dir, category)
    for file in tqdm(os.listdir(folder_path), desc=f"{category}"):
        try:
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img_resized = resize(img, (15, 15, 3))  # expected model format
            flat_data = img_resized.flatten()
            X.append(flat_data)
            y.append(label)
        except Exception as e:
            print(f"Hata {file} dosyasında: {e}")

not_empty: 100%|██████████| 3045/3045 [00:06<00:00, 443.94it/s]


In [4]:
X = np.array(X)
y = np.array(y) 
#turning our data to a numpy series 

In [5]:
# training our model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

RandomForestClassifier(random_state=42)

In [6]:
# and then saving it
os.makedirs("model", exist_ok=True)
with open("./model/model.p", "wb") as f:
    pickle.dump(model, f)

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# splittinn our data set with train and test part
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# retrained our model (not necessary but for a clean test/train split, well get better results if we do that retaining
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# predicting with test set
y_pred = model.predict(X_test)

# model metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       651
           1       1.00      1.00      1.00       567

    accuracy                           1.00      1218
   macro avg       1.00      1.00      1.00      1218
weighted avg       1.00      1.00      1.00      1218


Confusion Matrix:
 [[651   0]
 [  0 567]]
